<a href="https://colab.research.google.com/github/jhoths/MamographyFYP/blob/main/Mammography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import tensorflow as tf
import shutil as sh
import pandas as pd
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [11]:
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

/content


In [12]:
from google.colab import files
from os import environ

In [13]:
!pip install -q kaggle

In [14]:
# upload kaggle API key
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [15]:
# define kaggle config folder
! mkdir "./kaggle" && mv "./kaggle.json" "./kaggle/kaggle.json"
environ['KAGGLE_CONFIG_DIR'] = './kaggle'

# hide kaggle API key for other users
! chmod 600 ./kaggle/kaggle.json

mkdir: cannot create directory ‘./kaggle’: File exists


In [16]:
# fetch kaggle dataset
!kaggle datasets download -d vishumaa/breasy-cancer-binary
!unzip "./*.zip" && rm *.zip [y]

 99% 466M/471M [00:06<00:00, 145MB/s]
100% 471M/471M [00:06<00:00, 72.3MB/s]
Archive:  ./breasy-cancer-binary.zip
replace Breast_Cancer_Binary/benign/1-012.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping

# Define data directory
data_dir = '/content/Breast_Cancer_Binary'

# Define image dimensions and batch size
image_height = 512
image_width = 512
batch_size = 32

# Data preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generate data batches
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

# Define the model
model = Sequential([
    Conv2D(8, (3, 3), input_shape=(image_height, image_width, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(16, (3, 3)),
    BatchNormalization(),
    Activation('relu'),

    Conv2D(32, (3, 3)),
    BatchNormalization(),
    Activation('relu'),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classifier
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with both training and validation data
history = model.fit(train_generator, epochs=20,
                    validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print("Validation Accuracy:", accuracy)




Found 231 images belonging to 2 classes.
Found 57 images belonging to 2 classes.
Epoch 1/20
8/8 [==============================] - 140s 16s/step - loss: 170.8301 - accuracy: 0.4502 - val_loss: 0.8630 - val_accuracy: 0.5439
Epoch 2/20
8/8 [==============================] - 130s 16s/step - loss: 18.9645 - accuracy: 0.5671 - val_loss: 3.4100 - val_accuracy: 0.5439
Epoch 3/20
8/8 [==============================] - 125s 15s/step - loss: 9.1565 - accuracy: 0.7229 - val_loss: 3.5229 - val_accuracy: 0.5439
Epoch 4/20
8/8 [==============================] - 126s 15s/step - loss: 10.8440 - accuracy: 0.7229 - val_loss: 1.9484 - val_accuracy: 0.5439
Epoch 5/20
8/8 [==============================] - 134s 16s/step - loss: 4.7633 - accuracy: 0.8182 - val_loss: 5.4728 - val_accuracy: 0.5439
Epoch 6/20
2/2 [==============================] - 12s 3s/step - loss: 0.8630 - accuracy: 0.5439
Validation Accuracy: 0.5438596606254578
